# Overleaf to db file
input: main.tex Refs.bib 

In [5]:
import re
import pandas as pd

def parse_citations_from_tex(tex_file: str) -> pd.DataFrame:
    """
    Parse a LaTeX .tex file to extract citations, their frequency,
    and the section(s) they appear in.

    Parameters:
        tex_file (str): Path to the .tex file

    Returns:
        pd.DataFrame: A table with columns [Reference, Frequency, Sections]
    """
    # Read file and ignore comment lines
    with open(tex_file, "r", encoding="utf-8") as f:
        lines = f.readlines()

    # Remove commented lines (starting with %)
    clean_text = "\n".join(line for line in lines if not line.strip().startswith("%"))

    # Regex patterns
    section_pattern = re.compile(r'\\section\{([^}]*)\}(?:\\label\{[^}]*\})?')
    cite_pattern = re.compile(r'\\cite\{([^}]*)\}')

    # Split into sections
    sections = section_pattern.split(clean_text)

    citations = []
    ref_sections = {}

    # Iterate through sections
    for i in range(1, len(sections), 2):
        section_name = sections[i].strip()
        section_text = sections[i+1]

        # Find citations in this section
        matches = cite_pattern.findall(section_text)
        for match in matches:
            for key in match.split(","):
                ref = key.strip()
                citations.append(ref)
                if ref not in ref_sections:
                    ref_sections[ref] = []
                if section_name not in ref_sections[ref]:
                    ref_sections[ref].append(section_name)

    # Count frequencies while preserving order of first appearance
    freq = {}
    order = []
    for c in citations:
        if c not in freq:
            order.append(c)
        freq[c] = freq.get(c, 0) + 1

    # Build DataFrame
    df = pd.DataFrame({
        "Reference": order,
        "Frequency": [freq[c] for c in order],
        "Sections": [", ".join(ref_sections[c]) for c in order]
    })

    return df

import re
import pandas as pd

def parse_bibtex_to_dataframe(bib_file: str) -> pd.DataFrame:
    """
    Parse a .bib file and return a DataFrame with structured fields
    plus the full BibTeX entry.

    Parameters:
        bib_file (str): Path to the .bib file

    Returns:
        pd.DataFrame: Parsed bibliography
    """
    # Read the file
    with open(bib_file, "r", encoding="utf-8") as f:
        content = f.read()

    # Split into entries
    entries = ["@" + e for e in content.split("@") if e.strip()]

    papers = []

    for entry in entries:
        # Extract type and key
        match = re.match(r'@(\w+)\s*\{([^,]+),', entry)
        if not match:
            continue
        entry_type, entry_key = match.groups()

        # Extract fields (like title, author, year, etc.)
        fields = dict(re.findall(
            r'(\w+)\s*=\s*\{((?:[^{}]|\{[^}]*\})*)\}', 
            entry, 
            flags=re.DOTALL
        ))

        # Normalize important fields
        authors = fields.get("author", "").strip()
        title = fields.get("title", "").strip()
        journal = fields.get("journal", fields.get("booktitle", "")).strip()
        year = fields.get("year", "").strip()
        publisher = fields.get("publisher", fields.get("organization", "")).strip()

        papers.append({
            "Key": entry_key,
            "Type": entry_type,
            "Authors": authors,   # 👈 NEW column for full author names
            "Title": title,
            "Journal/Booktitle": journal,
            "Year": year,
            "Publisher": publisher,
            "BibTeX": entry.strip()  # full raw entry
        })

    # Build DataFrame, keeping unique keys
    df = pd.DataFrame(papers).drop_duplicates(subset="Key", keep="first").reset_index(drop=True)

    return df

def merge_citations_with_bib(main_text_df: pd.DataFrame, references_bib_df: pd.DataFrame) -> pd.DataFrame:
    """
    Merge LaTeX citations (from main.tex) with BibTeX records (from references.bib),
    keeping the same order as in the LaTeX file.
    """
    # Convert BibTeX records into a lookup dictionary by Key
    bib_lookup = references_bib_df.set_index("Key").to_dict(orient="index")

    merged_records = []
    for _, row in main_text_df.iterrows():
        key = row["Reference"]
        bib_info = bib_lookup.get(key, {})  # safely lookup
        merged_records.append({
            "Reference": key,
            "Frequency": row["Frequency"],
            "Sections": row["Sections"],
            "Type": bib_info.get("Type", ""),
            "Authors": bib_info.get("Authors", ""),   # 👈 ADD AUTHORS HERE
            "Title": bib_info.get("Title", ""),
            "Journal/Booktitle": bib_info.get("Journal/Booktitle", ""),
            "Year": bib_info.get("Year", ""),
            "Publisher": bib_info.get("Publisher", ""),
            "BibTeX": bib_info.get("BibTeX", "")
        })

    return pd.DataFrame(merged_records)


references_bib = parse_bibtex_to_dataframe("references.bib")
main_text = parse_citations_from_tex("main.tex")

merged_df = merge_citations_with_bib(main_text, references_bib)
# Add index column starting from 1
merged_df.insert(0, "Index", range(1, len(merged_df) + 1))

import sqlite3

# Save merged_df to SQLite
conn = sqlite3.connect("references.db")
merged_df.to_sql("references", conn, if_exists="replace", index=False)
conn.commit()



# Cross Ref

In [ ]:
import sqlite3
import requests
import time, random
from difflib import SequenceMatcher

def enrich_references_with_crossref(db_path="references.db", table="references", topn=3):
    """
    Sequentially enrich all rows in the SQLite DB with Crossref BibTeX and Title similarity.
    Always keeps the best candidate among top N results.
    Title_Similarity is stored as percentage (0–100).
    """
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()

    # Ensure columns exist
    cur.execute(f'PRAGMA table_info("{table}")')
    existing_cols = [col[1] for col in cur.fetchall()]
    if "Crossref_BibTeX" not in existing_cols:
        cur.execute(f'ALTER TABLE "{table}" ADD COLUMN Crossref_BibTeX TEXT;')
    if "Title_Similarity" not in existing_cols:
        cur.execute(f'ALTER TABLE "{table}" ADD COLUMN Title_Similarity REAL;')
    conn.commit()

    # Fetch all rows
    cur.execute(f'SELECT Reference, Title, Authors, "Journal/Booktitle", Year, Publisher FROM "{table}"')
    rows = cur.fetchall()

    for i, (ref, title, authors, journal, year, publisher) in enumerate(rows, start=1):
        print(f"\n[{i}/{len(rows)}] Processing Reference={ref}")

        if not title:
            print("⚠️ No title found, skipping.")
            continue

        # Build query string
        query = title
        if authors: query += f" {authors.split(',')[0]}"
        if journal: query += f" {journal}"
        if year: query += f" {year}"
        if publisher: query += f" {publisher}"

        url = f"https://api.crossref.org/works?query.bibliographic={requests.utils.quote(query)}&rows={topn}"

        crossref_bibtex, best_score = "", 0
        try:
            r = requests.get(url, timeout=15)
            r.raise_for_status()
            items = r.json().get("message", {}).get("items", [])

            if not items:
                print("⚠️ No Crossref results found.")
            else:
                best = None
                for item in items:
                    cr_title = item.get("title", [""])[0]
                    score = SequenceMatcher(None, title.lower(), cr_title.lower()).ratio()
                    if score > best_score:
                        best_score = score
                        best = item

                # Convert to percentage (0–100)
                best_score = round(best_score * 100, 2)

                if best:
                    doi = best.get("DOI", "")
                    if doi:
                        bibtex_r = requests.get(
                            f"https://doi.org/{doi}",
                            headers={"Accept": "application/x-bibtex"},
                            timeout=15
                        )
                        if bibtex_r.status_code == 200:
                            crossref_bibtex = bibtex_r.text.strip()

        except Exception as e:
            print(f"⚠️ Crossref fetch failed for {ref}: {e}")

        # Update DB immediately
        cur.execute(
            f'UPDATE "{table}" SET Crossref_BibTeX=?, Title_Similarity=? WHERE Reference=?',
            (crossref_bibtex, best_score, ref)
        )
        conn.commit()

        print(f"✅ Updated: Similarity={best_score}%, BibTeX length={len(crossref_bibtex)}")

        # Random delay
        sleep_time = random.uniform(2, 5)
        print(f"⏳ Waiting {sleep_time:.1f} seconds before next request...")
        time.sleep(sleep_time)

    conn.close()
    print("\n🎉 Finished updating all records.")


enrich_references_with_crossref("references.db", "references")


[1/30] Processing Reference=chen2020renewable
✅ Updated: Similarity=1.00, BibTeX length=390
⏳ Waiting 10.3 seconds before next request...

[2/30] Processing Reference=somu2021deep
✅ Updated: Similarity=1.00, BibTeX length=406
⏳ Waiting 13.4 seconds before next request...

[3/30] Processing Reference=hassan2016systematic
✅ Updated: Similarity=1.00, BibTeX length=517
⏳ Waiting 7.2 seconds before next request...

[4/30] Processing Reference=lin2022hybrid
✅ Updated: Similarity=1.00, BibTeX length=482
⏳ Waiting 8.3 seconds before next request...

[5/30] Processing Reference=10734172
✅ Updated: Similarity=1.00, BibTeX length=484
⏳ Waiting 9.1 seconds before next request...

[6/30] Processing Reference=al2023hybrid
✅ Updated: Similarity=1.00, BibTeX length=391
⏳ Waiting 14.1 seconds before next request...

[7/30] Processing Reference=kim2023novel
✅ Updated: Similarity=1.00, BibTeX length=393
⏳ Waiting 13.6 seconds before next request...

[8/30] Processing Reference=fekri2021deep
✅ Updated: S

# Abbreviation (Elsevier Publisher)

In [ ]:
import sqlite3
import pandas as pd
import re

def load_ltwa_mapping(csv_path: str) -> dict:
    df = pd.read_csv(csv_path, sep='\t', engine='python', dtype=str)
    df = df.dropna(subset=['ABBREVIATION'])
    df_en = df[df['LANGUAGES'].str.contains('English', case=False, na=False)]
    return {row['WORD'].lower(): row['ABBREVIATION'] for _, row in df_en.iterrows()}

def find_abbreviation(word: str, mapping: dict) -> str:
    key = word.lower()
    if key in mapping and mapping[key]:
        return mapping[key]
    dash_key = f"{key}-"
    if dash_key in mapping and mapping[dash_key]:
        return mapping[dash_key]
    truncated = key
    while len(truncated) > 1:
        truncated = truncated[:-1]
        tr_key = f"{truncated}-"
        if tr_key in mapping and mapping[tr_key]:
            return mapping[tr_key]
    return word

def abbreviate_journal_title(title: str, mapping: dict) -> str:
    tokens = re.split(r'(\W+)', title or "")
    return ''.join(
        find_abbreviation(tok, mapping) if re.match(r'\w+', tok) else tok
        for tok in tokens
    )

def add_journal_abbreviations(db_path="references.db", table="references", ltwa_csv="ltwa.tsv"):
    """
    Add a new column 'Journal_Abbrev' immediately after 'Journal/Booktitle'
    in the SQLite DB, filled with English LTWA abbreviations.
    """
    # Load abbreviation mapping (English only)
    mapping = load_ltwa_mapping(ltwa_csv)

    conn = sqlite3.connect(db_path)
    cur = conn.cursor()

    # Get schema of the current table
    cur.execute(f'PRAGMA table_info("{table}")')
    cols_info = cur.fetchall()
    col_names = [c[1] for c in cols_info]

    if "Journal_Abbrev" in col_names:
        print("ℹ️ Column Journal_Abbrev already exists, skipping schema change.")
    else:
        # Rebuild table with new column in correct place
        new_cols = []
        for name in col_names:
            new_cols.append(name)
            if name == "Journal/Booktitle":
                new_cols.append("Journal_Abbrev")
        col_defs = ", ".join(
            f'"{c}" TEXT' if c not in ("Frequency", "Year") else f'"{c}" INTEGER'
            for c in new_cols
        )

        # Create temp table
        cur.execute(f'CREATE TABLE "{table}_new" ({col_defs});')

        # Copy data into new table with empty Journal_Abbrev
        select_expr = ", ".join(
            [f'"{c}"' if c != "Journal_Abbrev" else "NULL" for c in new_cols]
        )
        cur.execute(f'INSERT INTO "{table}_new" SELECT {select_expr} FROM "{table}";')

        # Replace old table
        cur.execute(f'DROP TABLE "{table}";')
        cur.execute(f'ALTER TABLE "{table}_new" RENAME TO "{table}";')
        conn.commit()

    # Now fill the abbreviations
    cur.execute(f'SELECT Reference, "Journal/Booktitle" FROM "{table}"')
    rows = cur.fetchall()

    for ref, journal in rows:
        abbrev = abbreviate_journal_title(journal, mapping) if journal else ""
        cur.execute(
            f'UPDATE "{table}" SET Journal_Abbrev=? WHERE Reference=?',
            (abbrev, ref)
        )
    conn.commit()
    conn.close()
    print("✅ Journal abbreviations added next to Journal/Booktitle")

add_journal_abbreviations(
    db_path="references.db",
    table="references",
    ltwa_csv="ltwa.txt"   # path to your LTWA abbreviation list
)


ℹ️ Column Journal_Abbrev already exists, skipping schema change.
✅ Journal abbreviations added next to Journal/Booktitle


# Final step

In [8]:
import sqlite3
import re

def add_crossref_bibtex_with_abbrev(db_path="references.db", table="references"):
    """
    Create a new column 'Crossref_BibTeX_Abbrev' where:
      - BibTeX key is replaced by the 'Reference' column value
      - Journal name is replaced by the 'Journal_Abbrev' column value
    """
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()

    # Ensure column exists
    cur.execute(f'PRAGMA table_info("{table}")')
    existing_cols = [col[1] for col in cur.fetchall()]
    if "Crossref_BibTeX_Abbrev" not in existing_cols:
        cur.execute(f'ALTER TABLE "{table}" ADD COLUMN Crossref_BibTeX_Abbrev TEXT;')
        conn.commit()

    cur.execute(f'SELECT Reference, Journal_Abbrev, Crossref_BibTeX FROM "{table}"')
    rows = cur.fetchall()

    for ref, journal_abbrev, crossref_bib in rows:
        if not crossref_bib:
            continue

        new_bib = crossref_bib.strip()

        # --- 1) Replace BibTeX key safely ---
        try:
            start_brace = new_bib.index("{")
            first_comma = new_bib.index(",", start_brace)
            entry_type = new_bib[:start_brace]        # e.g. "@article"
            # rebuild entry start
            new_start = f"{entry_type}{{{ref},"
            # replace old start with new start
            new_bib = new_start + new_bib[first_comma+1:]
        except ValueError:
            # fallback if format is unexpected
            pass

        # --- 2) Replace journal field with abbreviation ---
        if journal_abbrev:
            new_bib = re.sub(
                r'(journal\s*=\s*\{)[^}]+(\})',
                rf'\1{journal_abbrev}\2',
                new_bib,
                flags=re.IGNORECASE
            )

        # Update DB
        cur.execute(
            f'UPDATE "{table}" SET Crossref_BibTeX_Abbrev=? WHERE Reference=?',
            (new_bib, ref)
        )

    conn.commit()
    conn.close()
    print("✅ Crossref_BibTeX_Abbrev fixed with correct keys and journals")

add_crossref_bibtex_with_abbrev("references.db", "references")


✅ Crossref_BibTeX_Abbrev fixed with correct keys and journals
